# Where to focus a marketing campaign?

## 📖 Background
You are a data analyst at a crowdfunding site. For the next quarter, your company will be running a marketing campaign. The marketing manager wants to target those segments that have donated the most in the past year. She turned to you to help her with her upcoming meeting with the CEO.

## 💾 The data
You have access to the following information:

#### Historic crowdfunding donations
- "category" - "Sports", "Fashion", "Technology", etc.
- "device" - the type of device used.
- "gender" - gender of the user.
- "age range" - one of five age brackets.
- "amount" - how much the user donated in Euros.

In [1]:
# pip install jupyter-dash

In [2]:
# import libraries
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects as go

import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash_table import DataTable, FormatTemplate
from dash.dependencies import Input, Output

C:\Users\Jungsun\AppData\Local\Temp/ipykernel_13844/1286775913.py:9: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Jungsun\AppData\Local\Temp/ipykernel_13844/1286775913.py:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\Jungsun\AppData\Local\Temp/ipykernel_13844/1286775913.py:11: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  from dash_table import DataTable, FormatTemplate


In [3]:
# import data
df = pd.read_csv('./crowdfunding.csv')
df.head()

,category,device,gender,age,amount
0,Fashion,iOS,F,45-54,61.0
1,Sports,android,M,18-24,31.0
2,Technology,android,M,18-24,39.0
3,Technology,iOS,M,18-24,36.0
4,Sports,android,M,18-24,40.0


In [4]:
# create category table
cat_tb = df.groupby('category').agg({'amount':['sum','mean','count']}).reset_index().\
            sort_values(by=('amount','sum'),ascending=False)
cat_tb.columns.droplevel() # remove multiindex column names
cat_tb.columns=['category','Total Amount of Donations', 'Average Amounts of Donations',
       'Number of Donations'] # reset the column names
cat_tb

,category,Total Amount of Donations,Average Amounts of Donations,Number of Donations
2,Games,165483.0,39.655643,4173
3,Sports,163528.0,39.130893,4179
4,Technology,162731.0,39.269064,4144
0,Environment,162376.0,39.710443,4089
1,Fashion,159952.0,39.271299,4073


In [5]:
# function for creating data table
def make_dt(data,money_col,d_col):
    # set up money formet 
    money_format = FormatTemplate.money(2)
    money_col=money_col
    d_col =d_col
    d_columns = [{'name':x,'id':x} for x in d_col]
    d_columns +=[{'name':x,'id':x, 'type':'numeric','format':money_format} for x in money_col]

    #create data table
    d_table = DataTable(columns=d_columns, data=data.to_dict('records'), sort_action='native',filter_action='native')
    return d_table

# make function for creating groupping data
def groupping(col):
    dt = df.groupby(col)['amount'].agg(['sum','mean','count'])
    return dt

In [6]:
# Dash layout 
app = JupyterDash(__name__) #jupyterlab  dash
app.layout = html.Div(
    children=[
        html.H1(
             # head title
            children="Crowdfunding Marketing", style={'textAlign':'center'}
        ),
        html.Div(
            children=[
                # 2nd title
                html.H4('Donation Analysis by Category'),
                html.Div(
                    # data table about number of donations , total and average amount of donations by category 
                    make_dt(cat_tb,['Total Amount of Donations','Average Amounts of Donations'],['category','Number of Donations']),
                    style={'width':'50%'}
                ),
                html.Br(),
                html.Div(
                    children=[
                        # add Dropdown-top n total donation by category bar plot
                        html.H4('Number of Top Total Donation Category'),
                        dcc.Dropdown(id='cat_top',
                                    options=[{'label':'Top2','value':'2'},
                                            {'label':'Top3','value':'3'},
                                            {'label':'Top4','value':'4'}]),
                        dcc.Graph(id='category_graph') # first figure(bar chart)
                    ],style={'width':'80%'} ),
                html.Div(                
                    children=[
                        html.Div(
                            children=[
                        # add dropdown total, average and count pie plot
                            html.H4('Donation by Device'),
                            dcc.Dropdown(id='device_dd',
                                    options=[{'label':'Percentage of Average Amount','value':'mean'},
                                            {'label':'Percentage of Doners','value':'count'}]),
                            dcc.Graph(id='device_graph') # second graph
                            ] ,style={'width':'60%'}#60 percentage of width 
                        ),
                        html.Div(
                            children=[
                            # add dropdown total, average and count pie plot
                            html.H4('Donation by Gender'),
                            dcc.Dropdown(id='gender_dd',
                                    options=[{'label':'Percentage of Average Amount','value':'mean'},
                                            {'label':'Percentage of Doners','value':'count'}]),
                            dcc.Graph(id='gender_graph') # third graph
                            ] 
                        ,style={'width':'60%'}#60 percentage of width 
                        )    
                    ]),
                html.Div(
                    children=[
                        html.H4('Donation by Age and Gender'),
                        dcc.Dropdown(id='age_dd',
                                     options=[{'label':'Amount and Percentage of Average','value':'mean'},
                                             {'label':'Amount and Percentage of Doner','value':'count'}]),
                        dcc.Graph(id='age_graph') # fourth graph
                    ]
                )
            ])

    ],style={'text-align':'center', 'display':'inline-block', 'width':'100%'})   

# first graph ,set the input and output of the callback to link dropdown to the graph
@app.callback(
    Output(component_id='category_graph',component_property='figure'),
    Input(component_id='cat_top',component_property='value')
)
        
def update_plot(input_num):
    copied_df = df.copy(deep=True)
    # conditional select by Number of top n category
    if input_num:
        num_filter = int(input_num)
        cat_groupping = copied_df.groupby('category')['amount'].sum().reset_index().sort_values(by='amount',ascending=False)
        cat_filter = cat_groupping[:num_filter].sort_values(by='amount')
        bar_fig = px.bar(cat_filter, y='category', x='amount', text='amount',orientation='h',\
                         title=f'Top{input_num} Total Donation(Euro) by Category')
        bar_fig.update_layout(xaxis=dict(title='Amount(Euro)'),yaxis=dict(title='Category'))
    else:
        cat_groupping = copied_df.groupby('category')['amount'].sum().reset_index().sort_values(by='amount')
        bar_fig = px.bar(cat_groupping, y='category', x='amount', text='amount',orientation='h',\
                         title=f' Total Donation(Euro) by Category')
        bar_fig.update_layout(xaxis=dict(title='Amount(Euro)'),yaxis=dict(title='Category'))
        
    return bar_fig

 # 2nd graph, set the input and ouput of the callback to link down to pie graph
@app.callback(  
    Output(component_id='device_graph',component_property='figure'),
    Input(component_id='device_dd',component_property='value')
)
def update_plot(input_val):
    # conditional select by Number of top n category
    value_filter='sum'
    if input_val:
        value_filter = input_val
    device_groupping = groupping('device')
    pie_fig = px.pie(device_groupping,names=device_groupping.index, values=value_filter,color=device_groupping.index,\
                 color_discrete_map={'iOS':'darkblue','android':'royalblue'},\
                     title=f' Percentage of {value_filter} by Device')
    pie_fig.update_traces(textposition='inside', textinfo='percent+label')  
    
    return pie_fig
        
 # 3rd graph, set the input and ouput of the callback to link down to pie graph
@app.callback(  
    Output(component_id='gender_graph',component_property='figure'),
    Input(component_id='gender_dd',component_property='value')
)
def update_plot(input_val):
    # conditional select by Number of top n category
    value_filter='sum'
    if input_val:
        value_filter = input_val
    gender_groupping = groupping('gender')
    pie_fig2 = px.pie(gender_groupping,names=gender_groupping.index, values=value_filter,color=gender_groupping.index,\
                      color_discrete_map={'F':'darkblue','M':'royalblue','U':'lightblue'},\
                      title=f' Percentage of {value_filter} by Gender')
    pie_fig2.update_traces(textposition='inside', textinfo='percent+label')  
    
    return pie_fig2

 # 4th graph, set the input and ouput of the callback to link down to funnel graph
@app.callback(  
    Output(component_id='age_graph',component_property='figure'),
    Input(component_id='age_dd',component_property='value')
)
def update_plot(input_val):
    # conditional select by Number of top n category
    value_filter='sum'
    if input_val:
        value_filter = input_val
    age_groupping = groupping(['age','gender']).reset_index(['gender'])
    funnel_graph=px.funnel(age_groupping,y=age_groupping.index,x=age_groupping[value_filter],color=age_groupping['gender'],\
                           color_discrete_map={'F':'darkblue','M':'royalblue','U':'lightblue'},\
                          title=f' Percentage of {value_filter} by Age and Gender')
    funnel_graph.update_traces(textposition='inside', textinfo='percent total + value') 
    
    return funnel_graph

# Run app and display result inline in the notebook
app.run_server(mode='inline',debug=True)

## 📝  Result

### Top 3 categories are Game, Sports and Technology
- The most donated category was Game followed by sport and  gap amount was less than 2000 Euros. The difference amount between the most and least donation was 5,531 Euro. 
- Sport had the largest donors, 4179, which was just 5 more people than Game donors. 
- There are not much difference among average amounts of donations of all categories, with less than a euro.

###  Most of donors use iOS 
- With 65.2% of total donation, those were iOS users and only 34.8% was android users. 
- There was no difference between average amount of donation by android and iOS users. 
- Same as the total amount analysis by device, the most of donors used iOS taking more than 65% of total donors.

###  No gender difference
- There was not much difference of total donation and the number of donors between male and female
- The average amounts of donations among gender were taking parts equally.

###  Most of donors are 18-24 
- Half of total donation are  donated by the yongest age range bracket. The rest of age brackets are relatively similar percentage with at most 1% difference. 
- There are not much gap among age brackets and gender in average amount of donations.
- With the same result, the most of donors are young age and the next most donors are 35-44 age bracket which takes 13% of total donors . 